In [19]:
# Populating the interactive namespace from numpy, pandas and datetime
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime

In [20]:
# reading the earthquakes data
all_earthquakes = pd.read_table('earthquakes/EMEC_data.txt', sep = ';', skiprows = 12, na_values = ' ')

In [21]:
from math import *
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees) usine haversine formula
    (which is said to be better for small distances)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # Radius of earth in kilometers is 6371
    dist = 6371000* c
    return dist

In [26]:
def declustering(all_earthquakes):
    
    """
    Finds the dependent earthquakes (fore- and aftershocks)
    using a method by Grünthal which is described in Burkhard and Grünthal (2009). 
    As an input receives the dataframe of the earthquake catalog. Returns: 1. the original
    dataframe with two additional columns: Declustering with True values for independent 
    earthquakes and the False values for the dependent earthquakes; and Decindex with 0 values
    for independent earthquakes and the indexes of the main earthquakes for the dependent earthquakes;
    2. dataframe of the independent earthquakes only.
    """

    # reading the values of the relevant columns from the table
    lat = all_earthquakes['latitude'].values
    lon = all_earthquakes['longitude'].values
    Mw = all_earthquakes['Mw '].values
    year = all_earthquakes['year'].values
    month = all_earthquakes['month'].values
    day = all_earthquakes['day'].values

    # converting the month and day to numeric values
    month = pd.to_numeric(month, errors = 'coerce')
    day = pd.to_numeric(day, errors = 'coerce')

    # filling nans in month and day with 6 and 15 respectfully (the middle of the year and the middle of the month)
    month = [6 if np.isnan(x) else int(x) for x in month]
    day = [15 if np.isnan(x) else int(x) for x in day]

    # defining all the earthquakes as independent (the declustring as True for every earthquake)
    declustering = (lat > 0)
    n = len(lat)
    i = 0
    decindex = 0*lat

    while True:

        if i > n - 1:
            break

        # calculating distance and time frames
        dR = np.exp(1.77 + np.sqrt(0.037 + 1.02*Mw[i]))
        if Mw[i] < 7.8:
            dTf = np.exp(-4.77 + np.sqrt(0.62 + 17.32*Mw[i]))
        else:
            dTf = np.exp(6.44 + 0.055*Mw[i])
        if Mw[i] < 6.6:
            dTa = np.exp(-3.95 + np.sqrt(0.62 + 17.32*Mw[i]))
        else:
            dTa = np.exp(6.44 + 0.055*Mw[i])

        # finding aftershocks
        d = 1
        while True:
            if i + d > n - 1:
                break

            # calculate the time difference between the current and the following (i+d) earthquake
            delta = datetime(year[i + d], month[i + d], day[i + d]) - datetime(year[i], month[i], day[i])
            timediff = delta.days

            # if the time difference is smaller than the time frame for the current magnitude 
            if timediff < dTa:

                # calculate the distance between the current and the following (i+d) earthquakes locations
                dist = haversine(lon[i], lat[i], lon[i + d], lat[i + d])

                # if the distance is smaller than the distance frame for the current magnitude, compare the magnitudes
                if dist/1000 < dR:

                    # if the magnitude of the following (i+d) earthquake is smaller than that of the current one 
                    if Mw[i] > Mw[i + d]:

                        # define the (i+d) earthquake as a dependent one (aftershock) 
                        # and save the index of the main earthquake
                        declustering[i + d] = False
                        decindex[i + d] = i

                # move to the next earthquake relative to the current one
                d = d + 1
            else:
                break    

        # finding foreshocks
        d = 1
        while True:
            if i - d < 0:
                break

            # calculate the time difference between the current and the previous (i-d) earthquake
            delta = datetime(year[i], month[i], day[i]) - datetime(year[i - d], month[i - d], day[i - d])
            timediff = delta.days

            # if the time difference is smaller than the time frame for the current magnitude 
            if timediff < dTf:

                # calculate the distance between the current and the previous (i-d) earthquakes locations
                dist = haversine(lon[i], lat[i], lon[i - d], lat[i - d])

                # if the distance is smaller than the distance frame for the current magnitude, compare the magnitudes
                if dist/1000 < dR:

                    # if the magnitude of the previous (i-d) earthquake is smaller than that of the current one 
                    if Mw[i] > Mw[i - d]:

                        # define the (i-d) earthquake as a dependent one (foreshock) 
                        # and save the index of the main earthquake
                        declustering[i - d] = False
                        decindex[i - d] = i

                # move to the next earthquake relative to the current one       
                d = d + 1
            else:
                break   

        # move to the next earthquake
        i = i + 1

    # add the declustering and the index columns to the dataframe
    all_earthquakes['Declustering'] = declustering
    all_earthquakes['Decindex'] = decindex

    # define a new filtered dataframe of the independent earthquakes only
    Declustered_earthquakes = all_earthquakes.iloc[:,:][all_earthquakes.Declustering == True]

    # droping the unnecessary columns
    Declustered_earthquakes = Declustered_earthquakes.drop(['Declustering', 'Decindex'], axis=1)
    
    return all_earthquakes, Declustered_earthquakes

In [30]:
# percentage of the removed earthquakes
(28327 - 8215)/28327*100

70.99939986585237

In [27]:
[a, b] = declustering(all_earthquakes)

In [28]:
a

,year,month,day,hour,minute,latitude,longitude,depth,intensity,M_orig,type,Mw,Mw_err,reference,Declustering,Decindex
0,1005,,,,,43.46,11.88,,6.5,4.9,w,4.9,0.34,CPTI08,True,0.0
1,1005,,,,,41.49,13.81,,7.5,5.4,w,5.4,0.3,CPTI04,True,0.0
2,1010,1,8,,,40.60,27.00,,9,7.4,w,7.4,0.5,Pap,True,0.0
3,1016,,,,,32.20,35.50,,8,5.5,L,5.5,,FA,True,0.0
4,1032,3,6,,,34.80,35.10,,,6.8,L,7.0,,FA,True,0.0
5,1033,12,5,,,32.50,35.50,,9.5,6.5,S,6.5,,KKP,True,0.0
6,1038,11,2,,,41.00,28.70,,7,6.7,w,6.7,0.5,Pap,True,0.0
7,1039,2,2,,,38.40,27.30,,8,6.8,w,6.8,0.5,Pap,True,0.0
8,1045,,,,,34.05,-5.00,,8,5.8,w,5.8,,Pel,True,0.0
9,1047,,,,,31.00,35.50,,9,6.5,S,6.5,,FA,True,0.0


In [29]:
b

,year,month,day,hour,minute,latitude,longitude,depth,intensity,M_orig,type,Mw,Mw_err,reference
0,1005,,,,,43.46,11.88,,6.5,4.9,w,4.9,0.34,CPTI08
1,1005,,,,,41.49,13.81,,7.5,5.4,w,5.4,0.3,CPTI04
2,1010,1,8,,,40.60,27.00,,9,7.4,w,7.4,0.5,Pap
3,1016,,,,,32.20,35.50,,8,5.5,L,5.5,,FA
4,1032,3,6,,,34.80,35.10,,,6.8,L,7.0,,FA
5,1033,12,5,,,32.50,35.50,,9.5,6.5,S,6.5,,KKP
6,1038,11,2,,,41.00,28.70,,7,6.7,w,6.7,0.5,Pap
7,1039,2,2,,,38.40,27.30,,8,6.8,w,6.8,0.5,Pap
8,1045,,,,,34.05,-5.00,,8,5.8,w,5.8,,Pel
9,1047,,,,,31.00,35.50,,9,6.5,S,6.5,,FA
